In [25]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [44]:
SUFFIX = '_MIN_FOUR_VARIABLES_DATASET_NR_'#'_recategorized'
FILE_TO_LOAD = 'CHES2019_experts.csv' # 'CHES2019_experts_recategorized_4l3c4r.csv' #' CHES2019_experts_recategorized.csv'#'CHES2019_experts.csv', 
FOLDER_PREFIX = '' #'base_data/' #'recategorized_4l3c4r/' # 
FOLDER_IMPUTE_PREFIX = 'imputed_data/'

In [48]:
if os.path.exists('../data/base_data/' + FOLDER_PREFIX) == False:
    os.mkdir('../data/base_data/' + FOLDER_PREFIX)
    print('New folder created')

if os.path.exists('../data/base_data/' + FOLDER_IMPUTE_PREFIX) == False:
    os.mkdir('../data/base_data/' + FOLDER_IMPUTE_PREFIX)
    print('New folder created for imputed data')

New folder created for imputed data


In [5]:
data_base = pd.read_csv('../data/' + FILE_TO_LOAD)

In [6]:
data_base.head()

,econ_interven,environment,redistribution,civlib_laworder,immigrate_policy,sociallifestyle,lrgen_factor,lrecon_factor,galtan_factor
0,5.0,5.0,8.0,6.0,7.0,5.0,5.0,5.0,6.0
1,0.0,5.0,1.0,7.0,5.0,8.0,NaN,3.0,9.0
2,5.0,5.0,8.0,6.0,3.0,5.0,4.0,5.0,5.0
3,5.0,5.0,8.0,6.0,6.0,5.0,6.0,7.0,9.0
4,2.0,5.0,8.0,9.0,10.0,8.0,6.0,7.0,9.0


In [7]:
data_base.describe()

,econ_interven,environment,redistribution,civlib_laworder,immigrate_policy,sociallifestyle,lrgen_factor,lrecon_factor,galtan_factor
count,3432.000000,3262.000000,3396.000000,3405.000000,3438.000000,3461.000000,3610.000000,3554.000000,3596.000000
mean,4.548951,5.188228,4.409894,5.151542,5.681792,4.473851,5.299446,4.952448,5.076752
std,2.632320,2.647456,2.533628,2.930842,2.936724,3.266881,2.582166,2.450809,3.106851
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000
50%,5.000000,5.000000,4.000000,5.000000,6.000000,4.000000,5.000000,5.000000,5.000000
75%,6.000000,7.000000,6.000000,8.000000,8.000000,7.000000,7.000000,7.000000,8.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [8]:
data_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3823 entries, 0 to 3822
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   econ_interven     3432 non-null   float64
 1   environment       3262 non-null   float64
 2   redistribution    3396 non-null   float64
 3   civlib_laworder   3405 non-null   float64
 4   immigrate_policy  3438 non-null   float64
 5   sociallifestyle   3461 non-null   float64
 6   lrgen_factor      3610 non-null   float64
 7   lrecon_factor     3554 non-null   float64
 8   galtan_factor     3596 non-null   float64
dtypes: float64(9)
memory usage: 268.9 KB


In [9]:
# Obtain an overview of missing values and respective removal
for i in range(0, 7):
    print(str(i) + ': ' + str(len(data_base.dropna(subset = data_base.iloc[:,:-3].columns.values,thresh = i))))

0: 3823
1: 3649
2: 3599
3: 3535
4: 3425
5: 3274
6: 2912


# Preprocessing
Procedure:
- remove all rows where NAN's occur for independent variables (0-5) # where more than 2 (>=3) NAN's occur for independent variables (0-5)
- Split the dataset into subsets according to to-be-predicted columns lrgen, lrecon, galtan
- Remove NA's and non-required columns in these sub-datasets for these columns
- Subsequently, split the datasets into validation and training-test dataset
- The remaining NA's of the independent variables will then be imputed using R

In [10]:
# remove rows with excessive number of NAN's - at least 4 values out of 6 must be provided
data_base = data_base.dropna(subset = data_base.iloc[:,:-3].columns.values,thresh = 4)

In [11]:
# select only columns with values and drop non-required columns
data_lrgen_base = data_base[data_base['lrgen_factor'].isnull()==False].drop(['lrecon_factor', 'galtan_factor'], axis=1)
data_lrecon_base = data_base[data_base['lrecon_factor'].isnull()==False].drop(['lrgen_factor', 'galtan_factor'], axis=1)
data_galtan_base = data_base[data_base['galtan_factor'].isnull()==False].drop(['lrecon_factor', 'lrgen_factor'], axis=1)

In [15]:
# parameters
test_size = 0.1 # out of total dataset
train_size = 0.8 # out of total dataset - test dataset

In [38]:
# definition of imputation
imputer = MissForest(class_weight = 'balanced', random_state = 320, criterion=('squared_error', 'gini')) #miss forest

# LRGEN

In [13]:
# lrgen-data
X = data_lrgen_base.iloc[:, :-1]
y = data_lrgen_base.iloc[:, -1]
prefix = 'data_lrgen_base_'

#X.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_total' + SUFFIX + '.csv')

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3385 entries, 0 to 3816
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   econ_interven     3321 non-null   float64
 1   environment       3169 non-null   float64
 2   redistribution    3277 non-null   float64
 3   civlib_laworder   3284 non-null   float64
 4   immigrate_policy  3295 non-null   float64
 5   sociallifestyle   3310 non-null   float64
dtypes: float64(6)
memory usage: 185.1 KB


In [55]:
# lrgen-data
X = data_lrgen_base.iloc[:, :-1]
y = data_lrgen_base.iloc[:, -1]
prefix = 'data_lrgen_base_'


for i in range(0,5):
    # In the first step we will split the data in training/validation and test dataset (10%)
    X_rem, X_test, y_rem, y_test = train_test_split(X,y, test_size=test_size, random_state=34+i)
    # Now since we want the train and valid size to be 80%/20%, 
    # we have to define train_size
    X_train, X_valid, y_train, y_valid = train_test_split(X_rem,y_rem, train_size=train_size, random_state=52+i)
    X_imputed = imputer.fit_transform(X_train, y_train)
    X_train_imputed = pd.DataFrame(X_imputed, columns = X_train.columns, index=y_train.index).round(0)
    X_imputed = imputer.fit_transform(X_valid, y_valid)
    X_valid_imputed = pd.DataFrame(X_imputed, columns = X_valid.columns, index=y_valid.index).round(0)
    X_imputed = imputer.fit_transform(X_test, y_test)
    X_test_imputed = pd.DataFrame(X_imputed, columns = X_test.columns, index=y_test.index).round(0)
    #print(len(X_train),len(y_train),len(X_valid),len(y_valid),len(X_test),len(y_test))
    X_train_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(i) + '.csv')
    y_train.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(i) + '.csv')
    X_valid_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(i) + '.csv')
    y_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(i) + '.csv')
    X_test_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(i) + '.csv')
    y_test.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(i) + '.csv')
    X_train.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_train' + SUFFIX + str(i) + '.csv')
    y_train.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_train' + SUFFIX + str(i) + '.csv')
    X_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_valid' + SUFFIX + str(i) + '.csv')
    y_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_valid' + SUFFIX + str(i) + '.csv')
    X_test.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_test' + SUFFIX + str(i) + '.csv')
    y_test.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_test' + SUFFIX + str(i) + '.csv')
    del(X_rem, X_test, y_rem, y_test, X_train, X_valid, y_train, y_valid, X_imputed, X_train_imputed, X_valid_imputed, X_test_imputed)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1

# LRECON

In [56]:
# lrecon-data
X = data_lrecon_base.iloc[:, :-1]
y = data_lrecon_base.iloc[:, -1]
prefix = 'data_lrecon_base_'

for i in range(0,5):
    # In the first step we will split the data in training/validation and test dataset (10%)
    X_rem, X_test, y_rem, y_test = train_test_split(X,y, test_size=test_size, random_state=34+i)
    # Now since we want the train and valid size to be 80%/20%, 
    # we have to define train_size
    X_train, X_valid, y_train, y_valid = train_test_split(X_rem,y_rem, train_size=train_size, random_state=52+i)
    X_imputed = imputer.fit_transform(X_train, y_train)
    X_train_imputed = pd.DataFrame(X_imputed, columns = X_train.columns, index=y_train.index).round(0)
    X_imputed = imputer.fit_transform(X_valid, y_valid)
    X_valid_imputed = pd.DataFrame(X_imputed, columns = X_valid.columns, index=y_valid.index).round(0)
    X_imputed = imputer.fit_transform(X_test, y_test)
    X_test_imputed = pd.DataFrame(X_imputed, columns = X_test.columns, index=y_test.index).round(0)
    #print(len(X_train),len(y_train),len(X_valid),len(y_valid),len(X_test),len(y_test))
    X_train_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(i) + '.csv')
    y_train.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(i) + '.csv')
    X_valid_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(i) + '.csv')
    y_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(i) + '.csv')
    X_test_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(i) + '.csv')
    y_test.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(i) + '.csv')
    X_train.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_train' + SUFFIX + str(i) + '.csv')
    y_train.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_train' + SUFFIX + str(i) + '.csv')
    X_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_valid' + SUFFIX + str(i) + '.csv')
    y_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_valid' + SUFFIX + str(i) + '.csv')
    X_test.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_test' + SUFFIX + str(i) + '.csv')
    y_test.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_test' + SUFFIX + str(i) + '.csv')
    del(X_rem, X_test, y_rem, y_test, X_train, X_valid, y_train, y_valid, X_imputed, X_train_imputed, X_valid_imputed, X_test_imputed)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3368 entries, 0 to 3816
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   econ_interven     3307 non-null   float64
 1   environment       3161 non-null   float64
 2   redistribution    3266 non-null   float64
 3   civlib_laworder   3269 non-null   float64
 4   immigrate_policy  3278 non-null   float64
 5   sociallifestyle   3293 non-null   float64
dtypes: float64(6)
memory usage: 184.2 KB


# GALTAN

In [57]:
# galtan-data
X = data_galtan_base.iloc[:, :-1]
y = data_galtan_base.iloc[:, -1]
prefix = 'data_galtan_base_'

for i in range(0,5):
    # In the first step we will split the data in training/validation and test dataset (10%)
    X_rem, X_test, y_rem, y_test = train_test_split(X,y, test_size=test_size, random_state=34+i)
    # Now since we want the train and valid size to be 80%/20%, 
    # we have to define train_size
    X_train, X_valid, y_train, y_valid = train_test_split(X_rem,y_rem, train_size=train_size, random_state=52+i)
    X_imputed = imputer.fit_transform(X_train, y_train)
    X_train_imputed = pd.DataFrame(X_imputed, columns = X_train.columns, index=y_train.index).round(0)
    X_imputed = imputer.fit_transform(X_valid, y_valid)
    X_valid_imputed = pd.DataFrame(X_imputed, columns = X_valid.columns, index=y_valid.index).round(0)
    X_imputed = imputer.fit_transform(X_test, y_test)
    X_test_imputed = pd.DataFrame(X_imputed, columns = X_test.columns, index=y_test.index).round(0)
    #print(len(X_train),len(y_train),len(X_valid),len(y_valid),len(X_test),len(y_test))
    X_train_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_train' + SUFFIX + str(i) + '.csv')
    y_train.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_train' + SUFFIX + str(i) + '.csv')
    X_valid_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_valid' + SUFFIX + str(i) + '.csv')
    y_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_valid' + SUFFIX + str(i) + '.csv')
    X_test_imputed.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'X_test' + SUFFIX + str(i) + '.csv')
    y_test.to_csv('../data/base_data/' + FOLDER_PREFIX + FOLDER_IMPUTE_PREFIX + prefix + 'y_test' + SUFFIX + str(i) + '.csv')
    X_train.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_train' + SUFFIX + str(i) + '.csv')
    y_train.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_train' + SUFFIX + str(i) + '.csv')
    X_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_valid' + SUFFIX + str(i) + '.csv')
    y_valid.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_valid' + SUFFIX + str(i) + '.csv')
    X_test.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'X_test' + SUFFIX + str(i) + '.csv')
    y_test.to_csv('../data/base_data/' + FOLDER_PREFIX + prefix + 'y_test' + SUFFIX + str(i) + '.csv')
    del(X_rem, X_test, y_rem, y_test, X_train, X_valid, y_train, y_valid, X_imputed, X_train_imputed, X_valid_imputed, X_test_imputed)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

In [53]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3387 entries, 0 to 3816
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   econ_interven     3325 non-null   float64
 1   environment       3175 non-null   float64
 2   redistribution    3277 non-null   float64
 3   civlib_laworder   3290 non-null   float64
 4   immigrate_policy  3296 non-null   float64
 5   sociallifestyle   3317 non-null   float64
dtypes: float64(6)
memory usage: 185.2 KB
